# Azure custom vision image recognition

You can train publish and use your image clasification model in Azure Custom Vision studio https://www.customvision.ai/
But also you can use API as below and check all steps in Custom Vision studio in paralel.
This lab based on https://learn.microsoft.com/en-us/azure/ai-services/custom-vision-service/quickstarts/image-classification?tabs=windows%2Cvisual-studio&pivots=programming-language-python

In [ ]:
%pip install azure-cognitiveservices-vision-customvision

In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

# Set the values of your computer vision endpoint and computer vision key
# as environment variables:
try:
    endpoint = os.environ["AZURE_AI_CUSTOM_VISION_ENDPOINT"]
    key = os.environ["AZURE_AI_CUSTOM_VISION_KEY"]
    ai_resource_id = os.environ["AZURE_AI_RESOURCE_ID"]
    print("AZURE_AI_CUSTOM_VISION_ENDPOINT is ", endpoint)
except KeyError:
    print("Missing endpoint, key or resource id. Set them before running this sample.")
    print("Set them before running this sample.")
    exit()

AZURE_AI_CUSTOM_VISION_ENDPOINT is  https://ai-services84921.cognitiveservices.azure.com/


In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": key})
trainer = CustomVisionTrainingClient(endpoint, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": key})
predictor = CustomVisionPredictionClient(endpoint, prediction_credentials)

In [8]:
publish_iteration_name = "classifyModel"

# Create a new project
print ("Creating project...", end="")
project_name = uuid.uuid4()
print (project_name)
project = trainer.create_project(project_name)

Creating project...e4b9ded3-5888-4855-b224-65ebb4924667


In [33]:
# tags for the project
orange_tag = trainer.create_tag(project.id, "orange")
banana_tag = trainer.create_tag(project.id, "banana")

In [34]:
# collect and tag images

base_image_location = os.path.join (os.getcwd(), "data", "training-images")
print(f"training images path: {base_image_location}")
print("Adding images...")

image_list = []

for filename in os.listdir(os.path.join(base_image_location, "banana")):
  with open(os.path.join (base_image_location, "banana", filename), "rb") as image_contents:
    image_list.append(ImageFileCreateEntry(name=filename, contents=image_contents.read(), tag_ids=[banana_tag.id]))

for filename in os.listdir(os.path.join(base_image_location, "orange")):
  with open(os.path.join (base_image_location, "orange", filename), "rb") as image_contents:
    image_list.append(ImageFileCreateEntry(name=filename, contents=image_contents.read(), tag_ids=[orange_tag.id]))

print(f"Added {len(image_list)} immages")

training images path: c:\DY\Programming\Azure-AI\Azure-AI\data\training-images
Adding images...
Added 29 immages


In [ ]:
# ulpoad collected images
upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

In [36]:
# first iteration of the model
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

Training...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Wa

In [ ]:
# Jupyter Notebook crashed here, instanses of Project and Itteration lost here,
# so for recover process and do not spend money for one more training
# I go to https://www.customvision.ai/ take project and iteration IDs and set them manualy
class Project:
    id = "7da200e5-e8f6-41b6-8e92-e7b9623a0e06"
project = Project()

class Iteration:
    id = "ab0f326e-e953-4f48-a8ff-ba433735bf89"
iteration = Iteration()

In [ ]:
#  Publish trained iteration as prediction model 
import os

prediction_resource_id = os.environ["AZURE_AI_RESOURCE_ID"]
publish_iteration_name = "Iteration 1"
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)

True

In [30]:
# Use published model for prediction
# here I will use an aple image, I suppose it shoud be clasified as "orange" with higher probability than "banana"
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": key})
predictor = CustomVisionPredictionClient(endpoint, prediction_credentials)

with open(os.path.join (os.getcwd(), "data", "test-images","1.jpg"), "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))

	orange: 68.70%
	banana: 32.40%
